In [ ]:
!pip install transformers

In [ ]:
from transformers import pipeline

model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
pipe = pipeline('text-classification', model=model_name,
                 tokenizer=model_name) 

In [ ]:
pipe("Verpackung gut, Produkt absolut unbrauchbar - geht zurück")

In [ ]:
from transformers import AutoTokenizer

model_name = 'nlptown/bert-base-multilingual-uncased-sentiment'
tokenizer = AutoTokenizer.from_pretrained(model_name, do_lower_case=True)

In [ ]:
from transformers import AutoModelForSequenceClassification
from scipy.special import softmax


# das Modell muss zum Tokenizer passen!
model = AutoModelForSequenceClassification.from_pretrained(
    model_name, 
    output_attentions = False,
    output_hidden_states = False # wir benötigen keine Embeddings
)
# hier evtl. model.cpu() einsetzen
model.cpu()

In [ ]:
import pandas as pd
at = pd.DataFrame([{ "text": "Heute ist ein furchtbarer Tag"}, {"text": "Heute ist ein wunderschöner Tag"}])
at

In [ ]:
from tqdm.auto import tqdm
import torch
device = torch.device("cpu")

input_ids = []
attention_masks = []

scores = []
for i in tqdm(range(len(at)//100 + 1)):
    input_ids = []
    attention_masks = []
    for t in at[i*100:(i+1)*100]["text"].map(str).values:
        encoded_dict = tokenizer.encode_plus(
                            t,
                            add_special_tokens = True,    # '[CLS]' und '[SEP]'
                            max_length = 64,
                            truncation = True,
                            padding='max_length',
                            return_attention_mask = True,  # Attention-Masks erzeugen
                            return_tensors = 'pt',         # pytorch-Tensoren als Ergebnis
                       )
        input_ids.append(encoded_dict['input_ids'])
        attention_masks.append(encoded_dict['attention_mask'])
        
    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)        
    
    with torch.no_grad():
        res = model(input_ids.to(device), attention_mask=attention_masks.to(device))
        for r in res[0].cpu().detach().numpy():
            scores.append(list(softmax(r)))

In [ ]:
pd.DataFrame(scores, columns=range(1,6))